#Pre-Processing :)

Memproses hasil csv agar bisa diolah menjadi model

Tugas:

1. lakukan eksplorasi data, lihat apakah ada data yang **missing/null/nan atau ambigu, dll**
2. deteksi bahasa di **judul dan abstract**, apakah ada data yang full bahasa asing, jika ada **pisahkan** data tersebut *(preprocessing dilanjut hanya untuk data b.indo atau b.indo yang campur sedikit dengan bahasa asing)*
3. beberapa abstract mengandung keywords, oleh karena itu **pisahkan keywords** tersebut dan pindahkan ke kolom keywords
4. lakukan **cleaning** text pada **judul dan abstract**, text hanya *alphabetnumerical*, hapus whitespace tetapi pertahankan angka dengan whitespace seperti 0,0 0.0, -0, - 0, 0%
5. buat kolom baru dengan nama **fulltext** yang berisi **judul, abstract, keywords** yang sudah di preprocessing
6. lakukan **stopword** untuk fulltext, buat kolom baru dengan nama **fulltext_clean**
7. lakukan **stemming** untuk _fulltext_clean, buat kolom baru dengan nama **_fulltext_clean_stemming**
8. lakukan **deteksi pos-tag** untuk _fulltext_clean, buat kolom baru dengan nama **_fulltext_clean_postag**
9. lakukan **augmentasi data synonim replacement** untuk fulltext_clean, buat kolom baru dengan nama **_fulltext_clean_synonim**
10. lakukan **augmentasi data random deletion** untuk fulltext_clean, buat kolom baru dengan nama **_fulltext_clean_deletion**

Keterangan:

* untuk nomer 1 sampai 6 dikerjakan secara individu
* untuk nomer 7 sampai 10 dilakukan berkelompok (dua orang untuk satu nomer, dibagi saja)
* lakukan tokenization atau preprocessing lainnya jika diperlukan
* beberapa preprocessing akan memakan waktu lama, jadi tidak apa-apa jika tidak semua data di preprocessing, misal [0-500] data

## Import data dari drive

In [ ]:
#install library langdetect
!pip install langdetect

In [ ]:
!pip install PySastrawi

In [ ]:
import pandas as pd
import numpy as np
import re

from numpy import NaN

# Python program to demonstrate langdetect
from langdetect import detect
from langdetect import detect_langs

from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [ ]:
df_real = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Scrapping Data/VersiBaru/2117051001_AbdilahAsyidiqi_extractdatapublikasi.csv')
# df = df_real.head(500)
df = df_real
df = df.drop(columns=['Unnamed: 0'])

## Exploratory Data Analysis (EDA)
apakah ada data yang missing/null/nan atau ambigu, dll??

In [ ]:
# Cek untuk memastikan apakah csv termuat kedalam dataframe
df.head(4)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37764 entries, 0 to 37763
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   divisions    33244 non-null  object
 1   title        37764 non-null  object
 2   abstract     29593 non-null  object
 3   subjects     37746 non-null  object
 4   publication  19004 non-null  object
 5   publisher    29695 non-null  object
 6   creators     37741 non-null  object
 7   date         29296 non-null  object
 8   keywords     962 non-null    object
 9   uri          37764 non-null  object
dtypes: object(10)
memory usage: 2.9+ MB


In [ ]:
#Cek ada berapa data yang NA
df.isna().sum()

divisions       4520
title              0
abstract        8171
subjects          18
publication    18760
publisher       8069
creators          23
date            8468
keywords       36802
uri                0
dtype: int64

In [ ]:
#Cek ada berapa data yang memiliki value
df.notna().sum()

divisions      33244
title          37764
abstract       29593
subjects       37746
publication    19004
publisher      29695
creators       37741
date           29296
keywords         962
uri            37764
dtype: int64

1. Setelah di cek, ternyata masih banyak data **Null/Kosong**
2. Kolom **title** dan **subjects** tidak memiliki value **NA**, sedangkan kolom lain terdapat value **NA**
3. Cek masing-masing value yang tidak null/NA dari tiap kolom

In [ ]:
divisions_aja = df['divisions']
pd.DataFrame(divisions_aja.dropna())

,divisions
0,['FT1']
1,['FKIP17']
2,['FP20']
3,['FH2']
4,['FKIP24']
...,...
37759,['FT1']
37760,['FT1']
37761,['FKIP9']
37762,['FKIP9']


In [ ]:
title_aja = df['title']
pd.DataFrame(title_aja.dropna())

,title
0,Determination of Volcanic Earthquake Hypocente...
1,PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...
2,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...
3,Principles of Islamic Sharia Economic Law in F...
4,Media Pembelajaran Digital sebagai Sumber Bela...
...,...
37759,Analysis of gravity anomaly for groundwater ba...
37760,Preliminary study of HVSR forward modeling: pa...
37761,The Effectiveness of Students’ Worksheet of ...
37762,The Effectiveness of Students’ Worksheet of ...


In [ ]:
abstract_aja = df['abstract']
pd.DataFrame(abstract_aja.dropna())

,abstract
0,Indonesia became an area passed by the Pacific...
1,The Effect of Mathematical Representation Skil...
2,Metil metsulfuron merupakan senyawa aktif yang...
3,Hasil Check Similarity
4,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
...,...
37752,"Dalam perkembangannya, karya cipta hasil kreas..."
37753,Indonesia is known for its cultural diversity ...
37756,Suoh merupakan daerah tektonik aktif yang meng...
37759,The research aims to delineate the groundwater...


4. Pada bagian abstract terdapat Kata kunci yang seharusnya masuk kebagian keyword

In [ ]:
subjects_aja = df['subjects']
pd.DataFrame(subjects_aja.dropna())

,subjects
0,['TN']
1,['L1']
2,"['Q1', 'SH']"
3,['K1']
4,['LA']
...,...
37759,"['QC', 'QE']"
37760,"['QE', 'T1']"
37761,['L1']
37762,['L1']


In [ ]:
publication_aja = df['publication']
pd.DataFrame(publication_aja.dropna())

,publication
1,Jurnal Pembelajaran Fisika
3,American Journal of Humanities and Social Scie...
4,EDUKATIF : JURNAL ILMU PENDIDIKAN
6,Heliyon
8,4th International Conference in Agroforestry 2022
...,...
37754,Research on Humanities and Social Sciences
37756,POSITRON Berkala Ilmiah Fisika
37759,IOP Conf. Series: Journal of Physics: Conf. Se...
37760,Journal of Physics: Conference Series


In [ ]:
publisher_aja = df['publisher']
pd.DataFrame(publisher_aja.dropna())

,publisher
1,Universitas Lampung
2,"Jurusan Perikanan dan Kelautan, Fakultas Perta..."
3,AJHSSR
4,Universitas Pahlawan Tuanku Tambusai
5,"Jurusan Pendidikan MIPA, FKIP Universitas Lampung"
...,...
37757,"Jurusan Teknik Geofisika, Fakultas Teknik, Uni..."
37758,"Jurusan Fisika, Fakultas Matematika dan Ilmu P..."
37760,IOP Publishing
37761,"International Journal of Advanced Engineering,..."


In [ ]:
date_aja = df['date']
pd.DataFrame(date_aja.dropna())

,date
0,3018
1,2107
2,2024-01-15 00:00:00
3,2023
4,2023-02-28
...,...
29291,20
29292,0017-04-17
29293,0016-03-22
29294,0016-05


5. Value pada tahun terdapat beberapa anomali seperti 2107, 3018, 0020-12, dll

In [ ]:
keyword_aja = df['keywords']
jumlah_sebelum_ekstrak = keyword_aja.notna().sum()
pd.DataFrame(keyword_aja.dropna())

,keywords
32,"State Space Model, Strategic Decision-Making, ..."
51,"batubara, Jalan, kecepatan, pengangkutan, prod..."
62,"Persimpangan, Geometrik, CBD, Traffic Light, A..."
63,"Kabupaten Pringsewu, Tingkat Ancaman, Ting..."
90,"Potential Spread of Covid-19, Error Ellipsoid ..."
...,...
36004,"cassava, starch content and strain gauge"
37032,"proyek, penjadwalan, keterlambatan, crash prog..."
37223,"Agribisnis ubi kayu, kemitraan, kesediaan peta..."
37245,"alat pengolahan tanah, mekanisasi pertanian, m..."


6. Format penulisan keyword **tidak menggunakan pola yang sama**, contohnya:

* Index ke-655 | *: Pantai Labuhan Jukung, gelombang, abrasi, fenomena, wisata*
* Index ke-2841	| *Kata kunci: Alih fungsi lahan, Hidroponik, Pertanian pekarangan, Rumah hijau*
* Index ke-4693	| *Keywords: city infrastructure, in-depth interview method, purposive sampling technique, sustainability, TBL criteria, TBL indicators.*
* Index ke-5562	| *Copper, Residue, Retainment, Tropical Soil, Waste, Zinc*
* Index ke-14008 | *x*
* Index ke-33962 | *xxxx*
* Index ke-36235 | *XXX*

## Bila terdapat kata kunci dibagian abstract, pindahkan ke kolom Keyword
7. Buat fungsi untuk mengekstrak kata kunci dari abstrak

In [ ]:
# Fungsi untuk mengekstrak kata kunci dari abstrak menggunakan regex
def extract_keywords(abstract):
    # Pola regex untuk mencari kata kunci
    keyword_pattern = r'(Keywords|Kata Kunci|key word).*(:|—)(.*$)'
    flags = re.IGNORECASE | re.MULTILINE
    try:
      match = re.search(keyword_pattern, abstract, flags)  # Mengabaikan huruf besar/kecil
    except:
      return np.nan
    if match:
      result = match.group(3).strip()
      if result == ("" or "—" or "-" or np.nan):
        return np.nan
      else:
        return result
    else:
      return np.nan

In [ ]:
# Fungsi untuk menghapus kata kunci dari abstrak menggunakan regex
def delete_keywords(abstract):
    # Pola regex untuk mencari kata kunci
    keyword_pattern = r'(?i)(Keywords|Kata Kunci|key word).*'
    try:
      flags = re.IGNORECASE | re.MULTILINE
      match = re.search(keyword_pattern, abstract, flags)  # Mengabaikan huruf besar/kecil
      if match:
         result = re.sub(keyword_pattern, '', abstract)
      else:
        result = abstract
      if result == ("" or "—" or "-" or np.nan):
        return np.nan
      else:
        return result
    except:
      result = abstract
      return result

8. Apply function ke kolom ***keywords***
9. Apply function ke kolom baru ***new abstract***

In [ ]:
# Terapkan fungsi delete_keywords & extract_keywords pada setiap baris DataFrame
new_abstract = []

for i in range(len(df['abstract'])):
  new_abstract.append(delete_keywords(df['abstract'][i]))
  if pd.isnull(df['keywords'][i]):
    df['keywords'][i] = extract_keywords(df['abstract'][i])

df['new_abstract'] = new_abstract

In [ ]:
# Tampilkan hasilnya
df

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Indonesia became an area passed by the Pacific...
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,The Effect of Mathematical Representation Skil...
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Hasil Check Similarity
4,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
...,...,...,...,...,...,...,...,...,...,...,...
37759,['FT1'],Analysis of gravity anomaly for groundwater ba...,The research aims to delineate the groundwater...,"['QC', 'QE']",IOP Conf. Series: Journal of Physics: Conf. Se...,NaN,"[{'name': {'lineage': None, 'given': 'ahmad', ...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,The research aims to delineate the groundwater...
37760,['FT1'],Preliminary study of HVSR forward modeling: pa...,The application of horizontal to a vertical sp...,"['QE', 'T1']",Journal of Physics: Conference Series,IOP Publishing,"[{'id': 'ahmad.zaenudin@eng.unila.ac.id', 'nam...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/3...,The application of horizontal to a vertical sp...
37761,['FKIP9'],The Effectiveness of Students’ Worksheet of ...,NaN,['L1'],NaN,"International Journal of Advanced Engineering,...","[{'id': None, 'name': {'honourific': None, 'fa...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,NaN
37762,['FKIP9'],The Effectiveness of Students’ Worksheet of ...,NaN,['L1'],NaN,"International Journal of Advanced Engineering,...","[{'id': None, 'name': {'lineage': None, 'given...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,NaN


In [ ]:
print(df['new_abstract'].notna().sum())
print(df['abstract'].notna().sum())
print(df['new_abstract'].isna().sum())
print(df['abstract'].isna().sum())

29593
29593
8171
8171


In [ ]:
print(df['keywords'].isna().sum())
print(df['keywords'].notna().sum())

29816
7948


In [ ]:
#simpan dataset dengan kolom keywords baru
df.to_csv("/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Dataset dengan keyword dan abstrak baru/df_with_new_keywords_and_abstract.csv", index=False)

## Buat raw Fulltext dari Title, New Abstract dan Keyword

10. Isi nilai NaN di Title, New Abstract dan Keyword menjadi string kosong
11. Buat function createRawFullText

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Dataset dengan keyword dan abstrak baru/df_with_new_keywords_and_abstract.csv')

In [ ]:
df

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Indonesia became an area passed by the Pacific...
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,The Effect of Mathematical Representation Skil...
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Hasil Check Similarity
4,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
...,...,...,...,...,...,...,...,...,...,...,...
37759,['FT1'],Analysis of gravity anomaly for groundwater ba...,The research aims to delineate the groundwater...,"['QC', 'QE']",IOP Conf. Series: Journal of Physics: Conf. Se...,NaN,"[{'name': {'lineage': None, 'given': 'ahmad', ...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,The research aims to delineate the groundwater...
37760,['FT1'],Preliminary study of HVSR forward modeling: pa...,The application of horizontal to a vertical sp...,"['QE', 'T1']",Journal of Physics: Conference Series,IOP Publishing,"[{'id': 'ahmad.zaenudin@eng.unila.ac.id', 'nam...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/3...,The application of horizontal to a vertical sp...
37761,['FKIP9'],The Effectiveness of Students’ Worksheet of ...,NaN,['L1'],NaN,"International Journal of Advanced Engineering,...","[{'id': None, 'name': {'honourific': None, 'fa...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,NaN
37762,['FKIP9'],The Effectiveness of Students’ Worksheet of ...,NaN,['L1'],NaN,"International Journal of Advanced Engineering,...","[{'id': None, 'name': {'lineage': None, 'given...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,NaN


In [ ]:
df['abstract'] = df['abstract'].replace(np.nan, "")
df['new_abstract'] = df['new_abstract'].replace(np.nan, "")
df['title'] = df['title'].replace(np.nan, "")
df['keywords'] = df['keywords'].replace(np.nan, "")

In [ ]:
df

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,,http://repository.lppm.unila.ac.id/id/eprint/5...,Indonesia became an area passed by the Pacific...
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,,http://repository.lppm.unila.ac.id/id/eprint/5...,The Effect of Mathematical Representation Skil...
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,,http://repository.lppm.unila.ac.id/id/eprint/5...,Hasil Check Similarity
4,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...
...,...,...,...,...,...,...,...,...,...,...,...
37759,['FT1'],Analysis of gravity anomaly for groundwater ba...,The research aims to delineate the groundwater...,"['QC', 'QE']",IOP Conf. Series: Journal of Physics: Conf. Se...,NaN,"[{'name': {'lineage': None, 'given': 'ahmad', ...",NaN,,http://repository.lppm.unila.ac.id/id/eprint/2...,The research aims to delineate the groundwater...
37760,['FT1'],Preliminary study of HVSR forward modeling: pa...,The application of horizontal to a vertical sp...,"['QE', 'T1']",Journal of Physics: Conference Series,IOP Publishing,"[{'id': 'ahmad.zaenudin@eng.unila.ac.id', 'nam...",NaN,,http://repository.lppm.unila.ac.id/id/eprint/3...,The application of horizontal to a vertical sp...
37761,['FKIP9'],The Effectiveness of Students’ Worksheet of ...,,['L1'],NaN,"International Journal of Advanced Engineering,...","[{'id': None, 'name': {'honourific': None, 'fa...",NaN,,http://repository.lppm.unila.ac.id/id/eprint/2...,
37762,['FKIP9'],The Effectiveness of Students’ Worksheet of ...,,['L1'],NaN,"International Journal of Advanced Engineering,...","[{'id': None, 'name': {'lineage': None, 'given...",NaN,,http://repository.lppm.unila.ac.id/id/eprint/2...,


In [ ]:
def createRawFulltext(data):
  return data['title'] + ' ' + data['new_abstract'] + ' ' + data['keywords']

In [ ]:
fulltext_raw = []
fulltext_raw = createRawFulltext(df)
df["text_raw"] = fulltext_raw

In [ ]:
fulltext_raw.head(5)

0    Determination of Volcanic Earthquake Hypocente...
1    PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...
2    TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...
3    Principles of Islamic Sharia Economic Law in F...
4    Media Pembelajaran Digital sebagai Sumber Bela...
dtype: object

In [ ]:
#simpan dataset dengan kolom fulltext baru
df.to_csv("/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Raw Fulltext/df_with_raw_fulltext.csv", index=False)

## Detect Language
Deteksi bahasa suatu jurnal dari kolom fulltext raw

In [ ]:
# deteksi akurasi bahasa

def language_detection_accuracy(column, index):
  acc_lang = detect_langs(column['text_raw'][index])
  return acc_lang

# deteksi bahasa
def language_detection(column, index):
  lang = detect(column['text_raw'][index])
  return lang

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Raw Fulltext/df_with_raw_fulltext.csv')

In [ ]:
df.head(5)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Indonesia became an area passed by the Pacific...,Determination of Volcanic Earthquake Hypocente...
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,The Effect of Mathematical Representation Skil...,PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Hasil Check Similarity,Principles of Islamic Sharia Economic Law in F...
4,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...


In [ ]:
acc_lang = []
lang = []

for i in range(len(df['text_raw'])):
  acc_lang.append(language_detection_accuracy(df,i))
  lang.append(language_detection(df,i))

In [ ]:
df['language'] = lang
df['language_accuracy'] = acc_lang

In [ ]:
df.head(5)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Indonesia became an area passed by the Pacific...,Determination of Volcanic Earthquake Hypocente...,en,[en:0.999996914004376]
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,The Effect of Mathematical Representation Skil...,PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,en,[en:0.999997164989931]
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,id,[id:0.9999985998246005]
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Hasil Check Similarity,Principles of Islamic Sharia Economic Law in F...,en,[en:0.9999973467127118]
4,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...,id,[id:0.9999984649495912]


In [ ]:
#simpan dataset dengan kolom baru yaitu 'language' dan 'language_accuracy'
df_with_lang = df
df.to_csv("/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Dataset dengan language/df_with_lang.csv", index= False)

In [ ]:
#Pisahkan data berbahasa indonesia dengan bahasa lain
df_id = df_with_lang[df['language'] == 'id']
df_other = df_with_lang[df['language'] != 'id']

In [ ]:
df_id.head(7)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy
2,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,id,[id:0.9999985998246005]
4,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...,id,[id:0.9999984649495912]
7,['FP1'],Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,['S1'],NaN,LPPM Universitas Lampung,"[{'name': {'given': 'Zainal', 'lineage': None,...",2023-09-19,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Persoalan penyediaan jasa air bersih merupakan...,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,id,[id:0.9999969177170068]
11,['FP17'],Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,['SF'],NaN,Kusuma Dokumen,"[{'name': {'honourific': None, 'family': 'Adhi...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,NaN,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,id,[id:0.999998385833847]
22,['FP25'],PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,['S1'],Jurnal Agrotropika,NaN,"[{'id': 'agustiansyah.1972@fp.unila.ac.id', 'n...",2023,"Bawang merah, Benih, Pelleting, TSS",http://repository.lppm.unila.ac.id/id/eprint/5...,Bawang merah merupakan komoditas pertanian pen...,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,id,[id:0.9999981965987415]
23,['FP8'],PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI ...,Tujuan kegiatan ini adalah mengenalkan \r\nkom...,['S1'],Jurnal Pengabdian Fakultas Pertanian Universit...,Fakultas Pertanian Universitas Lampung,"[{'name': {'honourific': None, 'given': 'Agust...",2023-03,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Tujuan kegiatan ini adalah mengenalkan \r\nkom...,PENGENALAN BUNGA KRISAN POTONG DI DESA SUNGAI ...,id,[id:0.9999986909923657]
26,['FP25'],EFEKTIVITAS PRIMING PADA BENIH JEGUNG DALAM ME...,Kandungan Aluminium yang tinggi pada tanah Ult...,['S1'],Jurnal Agrotropika,Jurnal Agrotropika,"[{'name': {'family': 'Agustin', 'lineage': Non...",2023-10-01,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Kandungan Aluminium yang tinggi pada tanah Ult...,EFEKTIVITAS PRIMING PADA BENIH JEGUNG DALAM ME...,id,[id:0.999998238387843]


In [ ]:
df_other.head(7)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy
0,['FT1'],Determination of Volcanic Earthquake Hypocente...,Indonesia became an area passed by the Pacific...,['TN'],NaN,NaN,"[{'id': 'suharno@eng.unila.ac.id', 'name': {'h...",3018,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Indonesia became an area passed by the Pacific...,Determination of Volcanic Earthquake Hypocente...,en,[en:0.999996914004376]
1,['FKIP17'],PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,The Effect of Mathematical Representation Skil...,['L1'],Jurnal Pembelajaran Fisika,Universitas Lampung,"[{'name': {'given': 'Illa', 'lineage': None, '...",2107,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,The Effect of Mathematical Representation Skil...,PENGARUH SKILL REPRESENTASI MATEMATIS TERHADAP...,en,[en:0.999997164989931]
3,['FH2'],Principles of Islamic Sharia Economic Law in F...,Hasil Check Similarity,['K1'],American Journal of Humanities and Social Scie...,AJHSSR,"[{'name': {'honourific': None, 'family': 'APRI...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Hasil Check Similarity,Principles of Islamic Sharia Economic Law in F...,en,[en:0.9999973467127118]
5,['FKIP15'],Check Smimilarity: STUDI PENDAHULUAN: PENGEMBA...,NaN,['L1'],NaN,"Jurusan Pendidikan MIPA, FKIP Universitas Lampung","[{'id': 'abdurrahman.1968@fkip.unila.ac.id', '...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,NaN,Check Smimilarity: STUDI PENDAHULUAN: PENGEMBA...,en,"[en:0.7142847424123915, tl:0.2857139264884598]"
6,['FKIP17'],Impacts of integrating engineering design proc...,"Currently, science education systems around th...",['L1'],Heliyon,ELSEVIER,"[{'name': {'honourific': None, 'family': 'Abdu...",2023-04-01,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,"Currently, science education systems around th...",Impacts of integrating engineering design proc...,en,[en:0.999996520844054]
8,NaN,Assessing Willingness to Pay for Improved Wate...,One of the important functions of the forest i...,['S1'],4th International Conference in Agroforestry 2022,IOP Conf. Series: Earth and Environmental Science,"[{'id': 'zainal.abidin@fp.unila.ac.id', 'name'...",2023-02-01,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,One of the important functions of the forest i...,Assessing Willingness to Pay for Improved Wate...,en,[en:0.999996752516443]
9,['FP1'],Similarity test IOP COnference 2023 assessing ...,Similarity test artikel Assessing WIllingness ...,['S1'],NaN,IOP Conf. Series: Earth and Environmental Science,"[{'id': 'zainal.abidin@fp.unila.ac.id', 'name'...",2023-02,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Similarity test artikel Assessing WIllingness ...,Similarity test IOP COnference 2023 assessing ...,en,[en:0.9999960380195168]


In [ ]:
df_id.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Dataset dengan language/id/df_id.csv', index= False)
df_other.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Dataset dengan language/other/df_other.csv', index= False)

## Cleaning
bersihkan tanda baca

In [ ]:
def clean_text(text):
  result = text.lower()
  result = re.sub(r'[^\w\s\.,%\-]+', ' ', result) # Remove Symbol
  result = re.sub(r'[_]+', ' ', result) # Remove Underscore
  result = re.sub(r'([\d]+)[\s]*(%)|([\d]+)[\s]*(-)[\s]*([\d]+)|(-)[\s]*([\d]+)', r'\1\2\3\4\5\6\7', result) # Remove space between digit and symbol (1 - 2, 1 %)
  result = re.sub(r'([\D])[\.,]+', r'\1 ', result) # Remove dot and comma (exceeding)
  result = re.sub(r'[\.,]+([\D])', r'\1 ', result) # Remove dot and comma (preceeding)
  result = re.sub(r'([\D])[%]+', r'\1 ', result) # Remove Percent
  result = re.sub(r'([\D])-+([\D])|-+([\D])', r'\1\2\3 ', result) # Remove Strip
  result = re.sub(r'[\s]+', ' ', result) # Remove Multiple Whitespace
  result = result.strip()
  return result

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Dataset dengan language/id/df_id.csv')

In [ ]:
df['fulltext'] = df['text_raw'].apply(lambda x: clean_text(x))

In [ ]:
df.head(5)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy,fulltext
0,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,id,[id:0.9999985998246005],toksisitas metil metsulfuron hubungannya denga...
1,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...,id,[id:0.9999984649495912],media pembelajaran digital sebagai sumber bela...
2,['FP1'],Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,['S1'],NaN,LPPM Universitas Lampung,"[{'name': {'given': 'Zainal', 'lineage': None,...",2023-09-19,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Persoalan penyediaan jasa air bersih merupakan...,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,id,[id:0.9999969177170068],strategi keberlanjutan jasa air bersih berbasi...
3,['FP17'],Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,['SF'],NaN,Kusuma Dokumen,"[{'name': {'honourific': None, 'family': 'Adhi...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,NaN,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,id,[id:0.999998385833847],metode sexing spermatozoa xy dan aplikasi inse...
4,['FP25'],PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,['S1'],Jurnal Agrotropika,NaN,"[{'id': 'agustiansyah.1972@fp.unila.ac.id', 'n...",2023,"Bawang merah, Benih, Pelleting, TSS",http://repository.lppm.unila.ac.id/id/eprint/5...,Bawang merah merupakan komoditas pertanian pen...,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,id,[id:0.9999981965987415],pengaruh bahan pelleting terhadap perkecambaha...


In [ ]:
df.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Fulltext Cleaned/df_fulltext_cleaned_id.csv', index=False)

## Stopword Removal

In [ ]:
def remove_stopword(text):
  factory = StopWordRemoverFactory()
  custom = [
    'mutlak',
    'era',
    'cenderung',
    'bantu',
    'paham',
    'dimana',
    'harap',
    'bahas',
    'definisi',
    'dimana',
    'dasar',
    'duga',
    'pakai',
    'pilih',
    'sesuai',
    'meliputi',
    'dituliskan',
    'dinyatakan',
    'keberlanjutan',
    'memperoleh',
    'diperoleh',
    'diharapkan',
    'mengharapkan',
    'berharap',
    'membahas',
    'dibahas',
    'didefinisikan',
    'mendefinisikan',
    'didalam',
    'kedalam',
    'memiliki',
    'dimiliki',
    'berdasarkan',
    'didasari',
    'berlandaskan',
    'diduga',
    'menduga',
    'menggambarkan',
    'digambarkan',
    'memakai',
    'dipakai',
    'memilih',
    'dipilih',
    'disesuaikan',
    'menyesuaikan',
    'kesesuaian',
    'diliputi',
    'menjabarkan',
    'dijabarkan',
    'penjabaran'
  ]
  stopwords = factory.get_stop_words()
  stopwords.extend(custom)

  result = []
  for x in  text.split():
    if x not in stopwords:
      result.append(x)

  return " ".join(result)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Fulltext Cleaned/df_fulltext_cleaned_id.csv')

In [ ]:
df['fulltext_stopword'] = df['fulltext'].apply(lambda x: remove_stopword(x))

In [ ]:
df.head(5)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy,fulltext,fulltext_stopword
0,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,id,[id:0.9999985998246005],toksisitas metil metsulfuron hubungannya denga...,toksisitas metil metsulfuron hubungannya masku...
1,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...,id,[id:0.9999984649495912],media pembelajaran digital sebagai sumber bela...,media pembelajaran digital sumber belajar maha...
2,['FP1'],Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,['S1'],NaN,LPPM Universitas Lampung,"[{'name': {'given': 'Zainal', 'lineage': None,...",2023-09-19,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Persoalan penyediaan jasa air bersih merupakan...,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,id,[id:0.9999969177170068],strategi keberlanjutan jasa air bersih berbasi...,strategi jasa air bersih berbasis masyarakat k...
3,['FP17'],Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,['SF'],NaN,Kusuma Dokumen,"[{'name': {'honourific': None, 'family': 'Adhi...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,NaN,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,id,[id:0.999998385833847],metode sexing spermatozoa xy dan aplikasi inse...,metode sexing spermatozoa xy aplikasi insemina...
4,['FP25'],PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,['S1'],Jurnal Agrotropika,NaN,"[{'id': 'agustiansyah.1972@fp.unila.ac.id', 'n...",2023,"Bawang merah, Benih, Pelleting, TSS",http://repository.lppm.unila.ac.id/id/eprint/5...,Bawang merah merupakan komoditas pertanian pen...,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,id,[id:0.9999981965987415],pengaruh bahan pelleting terhadap perkecambaha...,pengaruh bahan pelleting perkecambahan benih b...


In [ ]:
#Simpan file
df.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Stopword/df_fulltext_cleaned_stopword_id.csv', index = False)

## Stemming

In [ ]:
def stem(text):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  return stemmer.stem(text)

In [ ]:
df_steming = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Stopword/df_fulltext_cleaned_stopword_id.csv')

In [ ]:
df_steming.head(5)

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy,fulltext,fulltext_stopword
0,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,id,[id:0.9999985998246005],toksisitas metil metsulfuron hubungannya denga...,toksisitas metil metsulfuron hubungannya masku...
1,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...,id,[id:0.9999984649495912],media pembelajaran digital sebagai sumber bela...,media pembelajaran digital sumber belajar maha...
2,['FP1'],Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,['S1'],NaN,LPPM Universitas Lampung,"[{'name': {'given': 'Zainal', 'lineage': None,...",2023-09-19,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Persoalan penyediaan jasa air bersih merupakan...,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,id,[id:0.9999969177170068],strategi keberlanjutan jasa air bersih berbasi...,strategi jasa air bersih berbasis masyarakat k...
3,['FP17'],Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,['SF'],NaN,Kusuma Dokumen,"[{'name': {'honourific': None, 'family': 'Adhi...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,NaN,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,id,[id:0.999998385833847],metode sexing spermatozoa xy dan aplikasi inse...,metode sexing spermatozoa xy aplikasi insemina...
4,['FP25'],PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,['S1'],Jurnal Agrotropika,NaN,"[{'id': 'agustiansyah.1972@fp.unila.ac.id', 'n...",2023,"Bawang merah, Benih, Pelleting, TSS",http://repository.lppm.unila.ac.id/id/eprint/5...,Bawang merah merupakan komoditas pertanian pen...,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,id,[id:0.9999981965987415],pengaruh bahan pelleting terhadap perkecambaha...,pengaruh bahan pelleting perkecambahan benih b...


In [ ]:
df_steming.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15867 entries, 0 to 15866
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   divisions          14133 non-null  object
 1   title              15867 non-null  object
 2   abstract           12296 non-null  object
 3   subjects           15864 non-null  object
 4   publication        6874 non-null   object
 5   publisher          12214 non-null  object
 6   creators           15851 non-null  object
 7   date               12316 non-null  object
 8   keywords           3418 non-null   object
 9   uri                15867 non-null  object
 10  new_abstract       12296 non-null  object
 11  text_raw           15867 non-null  object
 12  language           15867 non-null  object
 13  language_accuracy  15867 non-null  object
 14  fulltext           15867 non-null  object
 15  fulltext_stopword  15867 non-null  object
dtypes: object(16)
memory usage: 1.9+ MB


In [ ]:
# Deklarasi variabel
range_data_per_part = []
pembagi = 500
jumlah_data = len(df)
hasil = 0

# Penentuan Part
jumlah_part = jumlah_data//pembagi
sisa = (jumlah_data%pembagi)
if sisa != 0:
  jumlah_part = jumlah_part+1

while(True):

  if hasil <= jumlah_data-pembagi:
    range_data_per_part.append(hasil)
    hasil = hasil + pembagi
    if hasil >= jumlah_data:
      range_data_per_part.append(hasil)
      break
  else:
    range_data_per_part.append(hasil)
    hasil = hasil + sisa
    range_data_per_part.append(hasil)
    break

In [ ]:
for i in range_data_per_part:
  print(i, end=' ')

print('\n')
print('Jumlah index part : ', len(range_data_per_part))
print('Jumlah data per-part : ', len(range_data_per_part)-1)
print('Jumlah data per-part yang seharusnya : ', jumlah_part, '(Pastikan sama dengan Jumlah data per-part)')

panjang_part = len(range_data_per_part)-1

0 500 1000 1500 2000 2500 3000 3500 4000 4500 5000 5500 6000 6500 7000 7500 8000 8500 9000 9500 10000 10500 11000 11500 12000 12500 13000 13500 14000 14500 15000 15500 15867 

Jumlah index part :  33
Jumlah data per-part :  32
Jumlah data per-part yang seharusnya :  32 (Pastikan sama dengan Jumlah data per-part)


In [ ]:
# Bagi data frame per part agar proses stemming bisa dipotong

df_steming_part = []
for j in range(len(range_data_per_part)):
  if j == panjang_part:
    break
  df_steming_part.append([])
  for i in range(range_data_per_part[j],range_data_per_part[j+1]):
    df_steming_part[j].append(df.iloc[i])

In [ ]:
# cek tiap part
pd.DataFrame(df_steming_part[31])

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy,fulltext,fulltext_stopword
15500,['FP5'],"PEER REVIEW JURNAL ""STRUKTUR PENDAPATAN, TINGK...",Program Hutan Kemasyarakatan (HKm) merupakan s...,['S1'],NaN,Penerbit Universitas Lampung with Indonesia Ne...,"[{'name': {'honourific': None, 'lineage': None...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/3...,Program Hutan Kemasyarakatan (HKm) merupakan s...,"PEER REVIEW JURNAL ""STRUKTUR PENDAPATAN, TINGK...",id,[id:0.9999956667708829],peer review jurnal struktur pendapatan tingkat...,peer review jurnal struktur pendapatan tingkat...
15501,NaN,Porang untuk Masyarakat Lampung,BUDI daya porang kali pertama\r\ndirintis oleh...,['S1'],https://radarlampung.co.id/porang-untuk-masyar...,NaN,"[{'id': 'gundowino@gmail.com', 'name': {'famil...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/3...,BUDI daya porang kali pertama\r\ndirintis oleh...,Porang untuk Masyarakat Lampung BUDI daya pora...,id,[id:0.9999948709715323],porang untuk masyarakat lampung budi daya pora...,porang masyarakat lampung budi daya porang dir...
15502,['FMIPA10'],Rancang Bangun Perpustakaan Digital,Berdasarkan surat edaran Dirjen Dikti No. 152/...,['QA75'],Jurnal Teknika,PPPM Politeknik Negeri Sriwijaya Palembang,"[{'name': {'family': 'Winata', 'given': 'Andri...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/3...,Berdasarkan surat edaran Dirjen Dikti No. 152/...,Rancang Bangun Perpustakaan Digital Berdasarka...,id,[id:0.9999963623538507],rancang bangun perpustakaan digital berdasarka...,rancang bangun perpustakaan digital surat edar...
15503,['FKIP16'],"Efikasi Diri, Lingkungan Keluarga dan Kebebasa...",Penelitian ini bertujuan untuk mengetahui peng...,['L1'],"Efikasi Diri, Lingkungan Keluarga dan Kebebasa...",FEB Universitas Muhammadiyah Pontianak,"[{'name': {'honourific': None, 'given': 'I Kom...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Penelitian ini bertujuan untuk mengetahui peng...,"Efikasi Diri, Lingkungan Keluarga dan Kebebasa...",id,[id:0.9999975033092267],efikasi diri lingkungan keluarga dan kebebasan...,efikasi lingkungan keluarga kebebasan pengaruh...
15504,NaN,Penatalaksanaan Penyakit Tuberkulosis dengan D...,Kelompok lanjut usia termasuk dalam populasi r...,['RA1001'],JK Unila JURNAL KEDOKTERAN UNIVERSITAS LAMPUNG,FK Unila,"[{'name': {'honourific': None, 'lineage': None...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/3...,Kelompok lanjut usia termasuk dalam populasi r...,Penatalaksanaan Penyakit Tuberkulosis dengan D...,id,[id:0.9999964228681321],penatalaksanaan penyakit tuberkulosis dengan d...,penatalaksanaan penyakit tuberkulosis diabetes...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15862,['FH3'],Perlindungan Terhadap Korban Perang \r\ndalam ...,Konflik bersenjata hingga kini masih menjadi p...,['K1'],NaN,ICRC,"[{'id': None, 'name': {'honourific': None, 'gi...",NaN,"Korban Perang, Hukum Humaniter Internasional, HAM",http://repository.lppm.unila.ac.id/id/eprint/2...,Konflik bersenjata hingga kini masih menjadi p...,Perlindungan Terhadap Korban Perang \r\ndalam ...,id,[id:0.9999968843606628],perlindungan terhadap korban perang dalam pene...,perlindungan korban perang penegakan hukum hum...
15863,['FH3'],HAK KEKAYAAN INTELEKTUAL BAGI PEKERJA SENI DI ...,"Dalam perkembangannya, karya cipta hasil kreas...",['K1'],Jurnal Pengabdian kepada Masyarakat Sakai Semb...,Lembaga Penelitian dan Pengabdian kepada Masya...,"[{'name': {'lineage': None, 'given': 'yunita m...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,"Dalam perkembangannya, karya cipta hasil kreas...",HAK KEKAYAAN INTELEKTUAL BAGI PEKERJA SENI DI ...,id,[id:0.9999969864056528],hak kekayaan intelektual bagi pekerja seni di ...,hak kekayaan intelektual pekerja seni sanggar ...
15864,['FT1'],Analisis Struktur Patahan Daerah Suoh Mengguna...,Suoh merupakan daerah tektonik aktif yang men

In [ ]:
print("Jumlah part yang dihasilkan : ",len(df_steming_part))

Jumlah part yang dihasilkan :  32


In [ ]:
#Lakukan proses steming per part. Atur berapa banyak data yang akan di stem pada bagian range
awal = 1
akhir = jumlah_part

for i in range(awal-1, akhir):
  df_steming_part_temp = pd.DataFrame(df_steming_part[i])
  df_steming_part_temp['text_stemmed'] = df_steming_part_temp['fulltext_stopword'].apply(lambda x : stem(x))

  #Simpan file
  df_steming_part_temp.to_csv(f'/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Stemming/df_fulltext_cleaned_stopword_stemming_id_part{i+1}.csv', index=False)
  print(f"Part {i+1} Sukses")

Part 1 Sukses
Part 2 Sukses
Part 3 Sukses
Part 4 Sukses
Part 5 Sukses
Part 6 Sukses
Part 7 Sukses
Part 8 Sukses
Part 9 Sukses
Part 10 Sukses
Part 11 Sukses
Part 12 Sukses
Part 13 Sukses
Part 14 Sukses
Part 15 Sukses
Part 16 Sukses
Part 17 Sukses
Part 18 Sukses
Part 19 Sukses
Part 20 Sukses
Part 21 Sukses
Part 22 Sukses
Part 23 Sukses
Part 24 Sukses
Part 25 Sukses
Part 26 Sukses
Part 27 Sukses
Part 28 Sukses
Part 29 Sukses
Part 30 Sukses
Part 31 Sukses
Part 32 Sukses


In [ ]:
# Import libraries
import glob

# Get CSV files list from a folder
path = '/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Stemming'
csv_files = glob.glob(path + "/*.csv")

# Read each CSV file into DataFrame
# This creates a list of dataframes
df_list = (pd.read_csv(file) for file in csv_files)

# Concatenate all DataFrames
df_hasil_stemming = pd.concat(df_list, ignore_index=True)

In [ ]:
df_hasil_stemming

,divisions,title,abstract,subjects,publication,publisher,creators,date,keywords,uri,new_abstract,text_raw,language,language_accuracy,fulltext,fulltext_stopword,text_stemmed
0,['FP20'],TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,Metil metsulfuron merupakan senyawa aktif yang...,"['Q1', 'SH']",NaN,"Jurusan Perikanan dan Kelautan, Fakultas Perta...","[{'name': {'honourific': None, 'lineage': None...",2024-01-15 00:00:00,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Metil metsulfuron merupakan senyawa aktif yang...,TOKSISITAS METIL METSULFURON HUBUNGANNYA DENGA...,id,[id:0.9999985998246005],toksisitas metil metsulfuron hubungannya denga...,toksisitas metil metsulfuron hubungannya masku...,toksisitas metil metsulfuron hubung maskulinit...
1,['FKIP24'],Media Pembelajaran Digital sebagai Sumber Bela...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,['LA'],EDUKATIF : JURNAL ILMU PENDIDIKAN,Universitas Pahlawan Tuanku Tambusai,"[{'id': 'suparman.arif@fkip.unila.ac.id', 'nam...",2023-02-28,NaN,http://repository.lppm.unila.ac.id/id/eprint/4...,Kegiatan pembelajaran siswa SMA dan SMK mutlak...,Media Pembelajaran Digital sebagai Sumber Bela...,id,[id:0.9999984649495912],media pembelajaran digital sebagai sumber bela...,media pembelajaran digital sumber belajar maha...,media ajar digital sumber ajar mahasiswa didik...
2,['FP1'],Strategi Keberlanjutan Jasa Air Bersih Berbasi...,Persoalan penyediaan jasa air bersih merupakan...,['S1'],NaN,LPPM Universitas Lampung,"[{'name': {'given': 'Zainal', 'lineage': None,...",2023-09-19,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,Persoalan penyediaan jasa air bersih merupakan...,Strategi Keberlanjutan Jasa Air Bersih Berbasi...,id,[id:0.9999969177170068],strategi keberlanjutan jasa air bersih berbasi...,strategi jasa air bersih berbasis masyarakat k...,strategi jasa air bersih bas masyarakat kawasa...
3,['FP17'],Metode Sexing Spermatozoa XY dan Aplikasi Inse...,NaN,['SF'],NaN,Kusuma Dokumen,"[{'name': {'honourific': None, 'family': 'Adhi...",2023,NaN,http://repository.lppm.unila.ac.id/id/eprint/5...,NaN,Metode Sexing Spermatozoa XY dan Aplikasi Inse...,id,[id:0.999998385833847],metode sexing spermatozoa xy dan aplikasi inse...,metode sexing spermatozoa xy aplikasi insemina...,metode sexing spermatozoa xy aplikasi insemina...
4,['FP25'],PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,Bawang merah merupakan komoditas pertanian pen...,['S1'],Jurnal Agrotropika,NaN,"[{'id': 'agustiansyah.1972@fp.unila.ac.id', 'n...",2023,"Bawang merah, Benih, Pelleting, TSS",http://repository.lppm.unila.ac.id/id/eprint/5...,Bawang merah merupakan komoditas pertanian pen...,PENGARUH BAHAN PELLETING TERHADAP PERKECAMBAHA...,id,[id:0.9999981965987415],pengaruh bahan pelleting terhadap perkecambaha...,pengaruh bahan pelleting perkecambahan benih b...,pengaruh bahan pelleting kecambah benih bawang...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15862,['FH3'],Perlindungan Terhadap Korban Perang \r\ndalam ...,Konflik bersenjata hingga kini masih menjadi p...,['K1'],NaN,ICRC,"[{'id': None, 'name': {'honourific': None, 'gi...",NaN,"Korban Perang, Hukum Humaniter Internasional, HAM",http://repository.lppm.unila.ac.id/id/eprint/2...,Konflik bersenjata hingga kini masih menjadi p...,Perlindungan Terhadap Korban Perang \r\ndalam ...,id,[id:0.9999968843606628],perlindungan terhadap korban perang dalam pene...,perlindungan korban perang penegakan hukum hum...,lindung korban perang tega hukum humaniter int...
15863,['FH3'],HAK KEKAYAAN INTELEKTUAL BAGI PEKERJA SENI DI ...,"Dalam perkembangannya, karya cipta hasil kreas...",['K1'],Jurnal Pengabdian kepada Masyarakat Sakai Semb...,Lembaga Penelitian dan Pengabdian kepada Masya...,"[{'name': {'lineage': None, 'given': 'yunita m...",NaN,NaN,http://repository.lppm.unila.ac.id/id/eprint/2...,"Dalam perkembangannya, karya cipta hasil kreas...",HAK KEKAYAAN INTELEKTUAL BAGI PEKERJA SENI DI ...,id,[id:0.9999969864056528],hak kekayaan intelektua

In [ ]:
df_hasil_stemming.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/df_final.csv', index = False)

## POS Tagging (ga kepake)

In [ ]:
!pip install python-crfsuite

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 8.9 MB/s eta 0:00:00


In [ ]:
!pip install nltk

In [ ]:
import nltk
from nltk.tag import CRFTagger
from nltk.tokenize import word_tokenize

import pycrfsuite

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
def pos_tag_word(word):
  token = word_tokenize(word)

  ct = CRFTagger()
  ct.set_model_file('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/requirements/all_indo_man_tag_corpus_model.crf.tagger')
  result = ct.tag_sents([token])

  return result[0]

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Stemming/df_fulltext_cleaned_stopword_stemming_id_part1.csv')

In [ ]:
df.head(5)

,Unnamed: 0,divisions,title,abstract,subjects,publication,publisher,date,keywords,new_abstract,fulltext_raw,language,language_accuracy,fulltext,fulltext_clean,text_stemmed
0,0,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963846268605],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru
1,1,NaN,Kewenangan Kejaksaan dalam Penyidikan dan Prap...,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Kewenangan Kejaksaan dalam Penyidikan dan Prap...,id,[id:0.9999962931569774],kewenangan kejaksaan dalam penyidikan dan prap...,kewenangan kejaksaan penyidikan prapenuntutan ...,wenang jaksa sidi prapenuntutan tindak pidana ...
2,2,NaN,Paradoks dan Epistimologi SAKSI MAHKOTA,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Paradoks dan Epistimologi SAKSI MAHKOTA,id,[id:0.9999961712909714],paradoks dan epistimologi saksi mahkota,paradoks epistimologi saksi mahkota,paradoks epistimologi saksi mahkota
3,3,['FK1'],Modul Pelatian TOT Bagi Dosen Home Based DLP,-,['R1'],NaN,BP SDM Kemenkes RI,NaN,NaN,-,Modul Pelatian TOT Bagi Dosen Home Based DLP -,id,"[id:0.7142826049168352, en:0.2857163348048405]",modul pelatian tot bagi dosen home based dlp,modul pelatian tot dosen home based dlp,modul pelatian tot dosen home based dlp
4,4,['FT8'],Peer Review - APLIKASI AUGMENTED REALITY NAVIG...,Aplikasi Augmented Reality Navigasi Koleksi Pe...,['T1'],NaN,Fakultas Teknik Universitas Lampung,NaN,NaN,Aplikasi Augmented Reality Navigasi Koleksi Pe...,Peer Review - APLIKASI AUGMENTED REALITY NAVIG...,id,[id:0.9999957742771008],peer review aplikasi augmented reality navigas...,peer review aplikasi augmented reality navigas...,peer review aplikasi augmented reality navigas...


In [ ]:
df['fulltext_clean_postag'] = df['fulltext_clean'].apply(lambda x: pos_tag_word(x))
df.head(5)

,Unnamed: 0,divisions,title,abstract,subjects,publication,publisher,date,keywords,new_abstract,fulltext_raw,language,language_accuracy,fulltext,fulltext_clean,text_stemmed,fulltext_clean_postag
0,0,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963846268605],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru,"[(jalan, NN), (sunyi, VB), (sang, NN), (guru, ..."
1,1,NaN,Kewenangan Kejaksaan dalam Penyidikan dan Prap...,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Kewenangan Kejaksaan dalam Penyidikan dan Prap...,id,[id:0.9999962931569774],kewenangan kejaksaan dalam penyidikan dan prap...,kewenangan kejaksaan penyidikan prapenuntutan ...,wenang jaksa sidi prapenuntutan tindak pidana ...,"[(kewenangan, NN), (kejaksaan, NN), (penyidika..."
2,2,NaN,Paradoks dan Epistimologi SAKSI MAHKOTA,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Paradoks dan Epistimologi SAKSI MAHKOTA,id,[id:0.9999961712909714],paradoks dan epistimologi saksi mahkota,paradoks epistimologi saksi mahkota,paradoks epistimologi saksi mahkota,"[(paradoks, NN), (epistimologi, NN), (saksi, N..."
3,3,['FK1'],Modul Pelatian TOT Bagi Dosen Home Based DLP,-,['R1'],NaN,BP SDM Kemenkes RI,NaN,NaN,-,Modul Pelatian TOT Bagi Dosen Home Based DLP -,id,"[id:0.7142826049168352, en:0.2857163348048405]",modul pelatian tot bagi dosen home based dlp,modul pelatian tot dosen home based dlp,modul pelatian tot dosen home based dlp,"[(modul, VB), (pelatian, NN), (tot, FW), (dose..."
4,4,['FT8'],Peer Review - APLIKASI AUGMENTED REALITY NAVIG...,Aplikasi Augmented Reality Navigasi Koleksi Pe...,['T1'],NaN,Fakultas Teknik Universitas Lampung,NaN,NaN,Aplikasi Augmented Reality Navigasi Koleksi Pe...,Peer Review - APLIKASI AUGMENTED REALITY NAVIG...,id,[id:0.9999957742771008],peer review aplikasi augmented reality navigas...,peer review aplikasi augmented reality navigas...,peer review aplikasi augmented reality navigas...,"[(peer, FW), (review, FW), (aplikasi, NN), (au..."


In [ ]:
df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
#Save file
df.to_csv(f'/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/POS Tagging/df_fulltext_clean_stopword_stemming_posttag_id_part1.csv')

# Augmentasi (Perbanyak data) | (ga kepake)

## Data Augmentation - Synonim

In [ ]:
!pip install tqdm

In [ ]:
import requests
import json
from tqdm import tqdm

In [ ]:
f = requests.get('https://raw.githubusercontent.com/victoriasovereigne/tesaurus/master/dict.json')
data = json.loads(f.text)

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/POS Tagging/df_fulltext_clean_stopword_stemming_posttag_id.csv")

In [ ]:
def replace_text(text):
  result = []
  text_token = text.split(' ')
  for i in range(len(text_token)):
    if text_token[i] in data.keys():
      for sinonim in data[text_token[i]]['sinonim']:
        temp = text_token.copy()
        temp[i] = sinonim
        result.append(' '.join(temp))

  return result

In [ ]:
def augment_synonim(data):
  result = pd.DataFrame()

  for i in replace_text(data['fulltext_clean']):
    temp_df = pd.DataFrame(data).T
    temp_df['fulltext_clean_synonim'] = i
    result = pd.concat([result, temp_df], ignore_index=True)

    temp_df

  return result

In [ ]:
df_augment = pd.DataFrame()
for i in tqdm(range(len(df))):
  df_augment = pd.concat([df_augment, augment_synonim(df.iloc[i])], ignore_index=True)

df_augment

100%|██████████| 233/233 [01:46<00:00,  2.19it/s]


,Unnamed: 0,divisions,title,abstract,subjects,publication,publisher,date,keywords,new_abstract,fulltext_raw,language,language_accuracy,fulltext,fulltext_clean,text_stemmed,fulltext_clean_postag,fulltext_clean_synonim
0,1,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963758162149],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru,"[('jalan', 'NN'), ('sunyi', 'VB'), ('sang', 'N...",adimarga sunyi sang guru
1,1,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963758162149],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru,"[('jalan', 'NN'), ('sunyi', 'VB'), ('sang', 'N...",bulevar sunyi sang guru
2,1,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963758162149],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru,"[('jalan', 'NN'), ('sunyi', 'VB'), ('sang', 'N...",denai sunyi sang guru
3,1,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963758162149],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru,"[('jalan', 'NN'), ('sunyi', 'VB'), ('sang', 'N...",gang sunyi sang guru
4,1,NaN,Jalan Sunyi Sang Guru,NaN,['K1'],NaN,Aura Publishing,NaN,NaN,NaN,Jalan Sunyi Sang Guru,id,[id:0.9999963758162149],jalan sunyi sang guru,jalan sunyi sang guru,jalan sunyi sang guru,"[('jalan', 'NN'), ('sunyi', 'VB'), ('sang', 'N...",jalan sunyi sang guru
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84500,499,['FMIPA4'],Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,Masyarakat Desa Legundi Kecamatan Ketapang Lam...,['Q1'],JURNAL PENGABDIAN KEPADA MASYARAKAT TABIKPUN,"Fakultas MIPA, UNILA",NaN,"Euchema sp, Masker Wajah, Pasta Gigi, Rumput Laut",Masyarakat Desa Legundi Kecamatan Ketapang Lam...,Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,id,[id:0.9999961249631903],pemanfaatan rumput laut euchema sp manjadi pas...,pemanfaatan rumput laut euchema sp manjadi pas...,manfaat rumput laut euchema sp manjadi pasta g...,"[('pemanfaatan', 'NN'), ('rumput', 'VB'), ('la...",pemanfaatan rumput laut euchema sp manjadi pas...
84501,499,['FMIPA4'],Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,Masyarakat Desa Legundi Kecamatan Ketapang Lam...,['Q1'],JURNAL PENGABDIAN KEPADA MASYARAKAT TABIKPUN,"Fakultas MIPA, UNILA",NaN,"Euchema sp, Masker Wajah, Pasta Gigi, Rumput Laut",Masyarakat Desa Legundi Kecamatan Ketapang Lam...,Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,id,[id:0.9999961249631903],pemanfaatan rumput laut euchema sp manjadi pas...,pemanfaatan rumput laut euchema sp manjadi pas...,manfaat rumput laut euchema sp manjadi pasta g...,"[('pemanfaatan', 'NN'), ('rumput', 'VB'), ('la...",pemanfaatan rumput laut euchema sp manjadi pas...
84502,499,['FMIPA4'],Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,Masyarakat Desa Legundi Kecamatan Ketapang Lam...,['Q1'],JURNAL PENGABDIAN KEPADA MASYARAKAT TABIKPUN,"Fakultas MIPA, UNILA",NaN,"Euchema sp, Masker Wajah, Pasta Gigi, Rumput Laut",Masyarakat Desa Legundi Kecamatan Ketapang Lam...,Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,id,[id:0.9999961249631903],pemanfaatan rumput laut euchema sp manjadi pas...,pemanfaatan rumput laut euchema sp manjadi pas...,manfaat rumput laut euchema sp manjadi pasta g...,"[('pemanfaatan', 'NN'), ('rumput', 'VB'), ('la...",pemanfaatan rumput laut euchema sp manjadi pas...
84503,499,['FMIPA4'],Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,Masyarakat Desa Legundi Kecamatan Ketapang Lam...,['Q1'],JURNAL PENGABDIAN KEPADA MASYARAKAT TABIKPUN,"Fakultas MIPA, UNILA",NaN,"Euchema sp, Masker Wajah, Pasta Gigi, Rumput Laut",Masyarakat Desa Legundi Kecamatan Ketapang Lam...,Pemanfaatan Rumput Laut (Euchema Sp) Manjadi P...,id,[id:0.9999961249631903],pemanfaatan rumput laut euchema sp manjadi pas...,pemanfaatan rumput laut euchema sp manjadi pas...,manfaat rumput laut e

In [ ]:
df = df_augment

In [ ]:
#Simpan file
df.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Augmentation - Synonim/df_fulltext_clean_stopword_stemming_posttag_sysnonim_id.csv')

## Augmentation - Random Deletion

In [ ]:
import random

In [ ]:
def delete_random(word):
  token = word_tokenize(word)
  random_element = random.choice(token)
  token.remove(random_element)

  return ' '.join(token)

In [ ]:
df_augment_random = df
df_augment_random['fulltext_clean_augment_deletion'] = df_augment_random['fulltext_clean'].apply(lambda x: delete_random(x))

In [ ]:
df_augment_random.head(10)

In [ ]:
#Simpan file
df_augment_random.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/Augmentation - Deletion/df_fulltext_clean_stopword_stemming_posttag_sysnonim_deletion_id.csv')

In [ ]:
#Simpan file
df_augment_random.to_csv('/content/drive/MyDrive/Dokumen/Tugas-Tugas/Kuliah/RISET MBKM-STKI/Preprocessing/df_final.csv')